In [1]:
# # -*- coding: utf-8 -*-
# from ckiptagger import data_utils # first use ckip, need to download data file
# data_utils.download_data_gdown("./")
from ckiptagger import WS, POS, NER, construct_dictionary

ws = WS("./ckip_data") # word segment
# pos = POS("./ckip_data") 
# ner = NER("./ckip_data") # entity recognition

import os
import csv
import pandas as pd
import numpy as np
import time
import math
import re
import demoji
# demoji.download_codes()


#====== import jieba dict =====
import jieba
jieba.set_dictionary('dataset/dic_trad/dict.txt.big')
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt')


#====== selenium activate esc key ======
from selenium.webdriver import ActionChains # activate esc key
from selenium.webdriver.common.keys import Keys # activate esc key

#====== selenium cathc NoSuchElementException ======
from selenium.common.exceptions import NoSuchElementException   
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#====== evaluation metrics ======
from sklearn.metrics import mean_squared_error #均方誤差
from sklearn.metrics import mean_absolute_error as mae #平方絕對誤差
from sklearn.metrics import r2_score

#====== MSE ======
def MSE(y, y_pred):
 
    mse = np.mean((y - y_pred)**2)
   
    return mse

/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/joung/anaconda3/envs/mlenv/lib/python3.6/site-


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



Building prefix dict from /home/alvin/essay/dataset/dic_trad/dict.txt.big ...
Loading model from cache /tmp/jieba.ucbd952ca8a83da26cb61ea55c46a2e3b.cache
Loading model cost 0.705 seconds.
Prefix dict has been built successfully.


In [2]:
#====== get antusd senti dict ======
colNames = ['Word','Score','Pos','Neu','Neg','Non','Not']
antusd = pd.read_csv('dataset/ANTUSD_traditional/opinion_word.csv', names = colNames) 
word_senti_score = dict(zip(antusd['Word'],antusd['Score'])) # find score via word, 27221 words

# print(antusd)
# print(word_senti_score)

In [112]:
len(antusd)

28799

In [4]:
#====== emoji_sentiment_dict ======
emoji_sentiment = pd.read_csv('dataset/dic_trad/Emoji_Sentiment.csv') # read Emoji_Sentiment.csv
emoji_sentiment.drop(columns=['Unnamed: 0'], inplace = True)
emoji_sentiment_dict = dict(zip(emoji_sentiment['Emoji'],emoji_sentiment['Sentiment score'])) # key,value

In [5]:
def get_selenium_param(filename):
    paramList = []
    with open("dataset/ig_article/"+filename, "r") as f:
        for line in f:
            single_store_param = line.split(',')
            single_store_param[2] = single_store_param[2].replace('\n','')
            paramList.append(single_store_param)

    return paramList

In [6]:
#====== data preprocess - related function ======
def get_store_names(food_kind):
    store_names = [food_kind]
    # read ranked store names
    with open("dataset/ig_article/store_article/"+food_kind+"/ranked_store_names", "r") as f:    
        for line in f:
            store_names.append(line.strip())
        
    return store_names
    
def get_remove_word():
    word_list = []
    with open("dataset/dic_trad/removed_word.txt", "r") as f:    
        for line in f:
            word_list.append(line.strip())
        
    return word_list

In [7]:
#====== extend senti. dict ======
def self_extend_senti_dict(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score


#====== extend senti. dict ======
def self_extend_senti_dict_hotpot(word_senti_score):
    with open("dataset/dic_trad/extend_senti_score_dict_hotpot.txt", "r") as f:
        for line in f:
            senti_word = line.split()[0]
            senti_score = float(line.split()[1])
            word_senti_score[senti_word] = senti_score    
        
#====== add emoji senti score ======
def add_emoji_senti_score(word_senti_score,emoji_sentiment_dict):
    word_senti_score.update(emoji_sentiment_dict)
#     return word_senti_score


self_extend_senti_dict(word_senti_score)
self_extend_senti_dict_hotpot(word_senti_score)
add_emoji_senti_score(word_senti_score, emoji_sentiment_dict)

In [8]:
emoji_sentiment_dict

{'😂': 0.221,
 '❤': 0.746,
 '♥': 0.657,
 '😍': 0.6779999999999999,
 '😭': -0.09300000000000001,
 '😘': 0.701,
 '😊': 0.644,
 '👌': 0.563,
 '💕': 0.632,
 '👏': 0.52,
 '😁': 0.449,
 '☺': 0.657,
 '♡': 0.669,
 '👍': 0.521,
 '😩': -0.368,
 '🙏': 0.41700000000000004,
 '✌': 0.46299999999999997,
 '😏': 0.332,
 '😉': 0.46299999999999997,
 '🙌': 0.5589999999999999,
 '🙈': 0.43200000000000005,
 '💪': 0.555,
 '😄': 0.42100000000000004,
 '😒': -0.374,
 '💃': 0.7340000000000001,
 '💖': 0.7120000000000001,
 '😃': 0.557,
 '😔': -0.146,
 '😱': 0.19,
 '🎉': 0.738,
 '😜': 0.455,
 '☯': 0.001,
 '🌸': 0.65,
 '💜': 0.654,
 '💙': 0.73,
 '✨': 0.35100000000000003,
 '😳': 0.018000000000000002,
 '💗': 0.657,
 '★': 0.28300000000000003,
 '█': -0.032,
 '☀': 0.465,
 '😡': -0.17300000000000001,
 '😎': 0.491,
 '😢': 0.006999999999999999,
 '💋': 0.691,
 '😋': 0.631,
 '🙊': 0.45899999999999996,
 '😴': -0.08,
 '🎶': 0.537,
 '💞': 0.7390000000000001,
 '😌': 0.48200000000000004,
 '🔥': 0.139,
 '💯': 0.12,
 '🔫': -0.19399999999999998,
 '💛': 0.7090000000000001,
 '💁': 0

In [9]:
print(len(word_senti_score),len(emoji_sentiment_dict))

28107 751


In [ ]:
#====== test ckip word segment ======

# text = '傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
# text = '"鬼金棒，我就跟！就這樣，我又跟我的前衣食父母吃飯！連吃兩天拉麵🍜一個胖的節奏#鬼金棒拉麵#辣正常麻增量#結果真的麻到我頭皮發麻 我到底要不要改機票呢😏"'
# text = '期盼那麼久才吃到，果然不負期望，無論是湯頭還是肉片都是最佳狀態，太享受了'
text = '好久不見！，體力一年不如一年，#20191101，服務態度超優的，海鮮😋,海底撈圈字遊戲送一盤肉,👍👍👍撈撈撈，💋💋💋,排隊排好久 超值得, 我要再來刷二訪 😋， 以愛吃重口味的我來說 麵條的硬度很讚 乾, 想吃很膩害的溏心蛋 這家沾麵真的很優,湯頭也很濃 我不愛沾麵 難波萬台灣美食成為我的愛店之一👍🏻好多人，不知道要排多久,麻的味道超級棒,推推推,湯頭還是很棒，豆芽菜的鑊氣沒了很難過，大概一陣子不會回訪了,good excelent,熱死第一名,可惜還是沒把湯喝完,應該很普通,不是我的最愛。'


ws_results = ws([text])
# pos_results = pos(ws_results)
# ner_results = ner(ws_results, pos_results)

print(ws_results)
# print(pos_results)
# for name in ner_results[0]:# print entity
#     print(name)

In [6]:
# word_pos = dict(zip(ws_results[0],pos_results[0])) # combine word and pos tag

In [26]:
#====== data preprocess - implementation ======
def data_preprocess(articles): 
    # only get string comment & preprocess fans, likes
    all_comments, assigned_index, fans_count, likes  = [],[],[],[]  # assigned_index: denote the original dataframe index of comments
    for i in range(len(articles['text'])):     # remove '\n'
        if type(articles['text'][i]) == str:   # remove non-string
            
            # exclude noise article(ex:染髮)
            temp_word_list = jieba.lcut(articles['text'][i])
            remove_flag = False
            for removed_word in get_remove_word():
                if removed_word in temp_word_list:
                    remove_flag = True
                    break
            if remove_flag:
                continue
                    
            # if contain '\n', replace with ','
            if articles['text'][i].find('\n'): 
                all_comments.append(articles['text'][i].replace('\n','，'))
            else:
                all_comments.append(articles['text'][i])
            assigned_index.append(i)
            
#             # get assigned fans_count
#             if type(articles['fans_count'][i]) == np.float64 and math.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif type(articles['fans_count'][i]) == np.float64 and np.isnan(articles['fans_count'][i]):
#                 fans_count.append(0)
#             elif str(articles['fans_count'][i]).find('萬') != -1:
#                 wan_index = articles['fans_count'][i].find('萬')
#                 transform_count = (float(articles['fans_count'][i][:wan_index])) * 10000
#                 fans_count.append(transform_count)
#             else:
#                 fans_count.append(int(articles['fans_count'][i]))
            fans_count = []


            # get assigned likes
            temp_like = articles['likes'][i]
            if type(temp_like) == np.float64 or type(temp_like) == np.int64:
                likes.append(temp_like)
            elif type(temp_like) == str:
                if temp_like.isdigit():
                    likes.append(float(temp_like))
                else:
                    likes.append(1)
            else:
                likes.append(temp_like)
            
        # get article at most 120 post
        if len(all_comments) == 150:
            break
    
    return all_comments, assigned_index, fans_count, likes

In [151]:
#====== cal senti score by jieba ======
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

# def get_senti_score(text):
#     '''calculate one sentence's score'''
#     ws_results = [jieba.lcut(text)] # word segment
#     # get sentiword in dict
#     total_score = 0
#     for word in ws_results[0]:
#         if word in word_senti_score:
#             score = word_senti_score[word]
#             total_score += score 
#             print(word,score) # print each senti word's senti score
    
#     # if comment's senti score over 1, return 1
#     print('sent_score:',total_score)
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score
    
def get_senti_score(text, print_comment = False):
    '''calculate one sentence's score'''
    ws_results = [jieba.lcut(text)] # word segment
    # get sentiword in dict
    total_score = 0
    for word in ws_results[0]:
        if word in word_senti_score:
            score = word_senti_score[word]
            total_score += score 

#     print('sent_score:',total_score)
    #
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score
    
    return total_score

In [ ]:
#====== cal senti score by ckip ======
# def get_senti_score(text):
#     ws_results = ws([text]) # word segment
#     total_score = 0
#     for word in ws_results[0]:
#         if word in word_senti_score:
#             score = word_senti_score[word]
#             total_score += score 
#             print(word,score) # print each senti word's senti score
    
    
#     print('sent_score:',total_score)
#     if total_score >= 1:
#         return 1
#     elif total_score <= -1:
#         return -1
#     else:
#         return total_score

In [80]:
#====== get each comment's score ======
def get_each_comment_score(all_comments, print_comment = True):
    all_comments_scores = []
    if print_comment:
        for single_comment in all_comments:
            print('\ncomment:',single_comment)
            all_comments_scores.append(get_senti_score(single_comment))
    else:
        for single_comment in all_comments:
            all_comments_scores.append(get_senti_score(single_comment, False))
        
    return all_comments_scores

In [15]:
#====== cal store score ======      # not yet consider no comment post's effect (maybe effective) 
def get_store_score(weight, each_comment_score, assigned_index):
    '''store score = comment score * likes(weight) /all likes'''
    weighted_score = 0
    total_weight = 0
    for i in range(len(each_comment_score)):
        if not np.isnan(weight[i]):
            weighted_score += (weight[i] * each_comment_score[i])
            total_weight += weight[i] 
            
    store_score = weighted_score / total_weight

    return store_score


In [18]:
def output_csv(filename, data_list, write_method):

    import csv

    # 開啟輸出的 CSV 檔案
    with open(filename, write_method, newline='') as csvfile:
        writer  = csv.writer(csvfile)   # 建立 CSV 檔寫入器
        for row in data_list: # format: two dimension
            writer.writerow(row)
            

In [ ]:
#====== remove foreign language post ======
# for i,content in zip(articles[articles['text'].notnull()]['text'].index,
#                      articles[articles['text'].notnull()]['text']):
#     if not re.search(r'[\u4e00-\u9fff]+', content) and not demoji.findall(content): # check contain chinese
#         print(i,content)
#         # drop assigned row
#         articles.drop(i, inplace=True) 
        
# # reset dataframe index
# articles.reset_index(drop=True) 

In [19]:
text = '這是一場五感都被放大的饗宴。﻿，﻿，有硬度及Q勁的麵體，🐠很喜歡，搭配辣少麻少的濃厚湯頭，非常過癮！鬼金棒的辣和麻是一流的，入口先感受到辣，隨之而來的麻會讓你感受到舌尖的震顫，建議第一次吃、不是很敢吃麻辣的朋友跟🐠一樣選擇辣少麻少。﻿，﻿，鬼金棒的拉麵，就是這樣強勁有力。﻿，﻿，叉燒肉也和大多數的拉麵店不同，選用大塊的五花肉，乍看之下有點像控肉（但絕對不是），肥瘦相間、軟嫩好入口，有別於一般薄片叉燒，鬼金棒就是這樣的霸氣！﻿，﻿，最後額外提一下，店裡的冰開水不知道為什麼，特別甘甜好喝啊🤣﻿，﻿，﻿，-·-·-·-·-·-﻿，🔺用餐請遵守店內顧客須知，大家一起當優良好客人！﻿，🔺吃完麵再加白飯100分！但麵的份量本身就蠻大了，要自己斟酌哦！﻿，——————————，﻿，🌟如果喜歡濃厚拉麵，或者吃膩了一般口味的拉麵，來鬼金棒走一遭吧☺️﻿，﻿，——————————，🏠辣麻味噌拉麵 鬼金棒﻿，📍104台北市中山區長安西路19巷2弄35號（近捷運中山站）﻿，🕐平日11:30-14:00/17:00-20:30，假日11:30-20:30﻿，，#辣麻味噌拉麵 #鬼金棒 #鬼金棒カラシビ味噌ラーメン #拉麵 #叉燒肉 #捷運美食 #捷運中山站 #ramen #ramennoodles 當然是想到鬼金棒💕，麻的味道超級棒，讓你一口接一口🤤，肉的份量也很足👍，湯頭非常濃郁‼️，吃#nightmarket #taiwan #taipei #noodles #stinkytofu 🤤#squid #seafood #dumplings #ningxia #nightlife,豪豪ㄘ 嘴唇超麻～～～,敲好ㄘ🌶🌶🌶🌶🍜'

# print(ws([text]))
demoji.findall(text)
for emoji in demoji.findall(text):
    if emoji_sentiment_dict.get(emoji):
        print(emoji,emoji_sentiment_dict[emoji])

🏠 0.5
👍 0.521
📍 0.111
🐠 0.414
🌟 0.327
🍜 0.395
💕 0.632


In [ ]:
jieba.lcut('新娘美甲喝起來更香濃愛吃重口味的我來說嗚嗚嗚嗚可惜吃到後面有點膩今天沾麵日！白湯&海湯沾麵🍜')

In [132]:
#======evaluation: read other rank method ======
def read_other_rank():
    rank_dict = {'google':{},'ifoodie':{}}
    food_kind = ['pasta','hotpot','ramen','beef_noodle','pork_rice']
    with open("dataset/ig_article/store_article/other_rank_method",'r') as file:
        for line in file:
            if line.find('google') != -1:        # not found, return -1
                for food in food_kind:
                    if line.find(food) != -1:     # ex:find google pasta rank
                        str_rank_list = line.strip().split(':')[1].split(',')
                        rank_dict['google'][food] = [int(i) for i in str_rank_list]
                        break
                    
                    
                    
#                 if line.find('pasta') != -1:     # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['pasta'] = [int(i) for i in str_rank_list]
                
#                 elif line.find('hotpot') != -1:   # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['hotpot'] = [int(i) for i in str_rank_list]
                
#                 elif line.find('ramen') != -1:   # find google pasta rank
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['google']['ramen'] = [int(i) for i in str_rank_list]
                    
            elif line.find('ifoodie') != -1:
#                 if line.find('pasta') != -1:
#                     str_rank_list = line.strip().split(':')[1].split(',')
#                     rank_dict['ifoodie']['pasta'] = [int(i) for i in str_rank_list]
                for food in food_kind:
                    if line.find(food) != -1:     # ex:find google pasta rank
                        str_rank_list = line.strip().split(':')[1].split(',')
                        rank_dict['ifoodie'][food] = [int(i) for i in str_rank_list]
                        break    
                    
                    
        
    return rank_dict
            
rank_dict = read_other_rank()
print(rank_dict)

{'google': {'ramen': [10, 4, 6, 4, 1, 7, 7, 1, 9, 1], 'hotpot': [9, 2, 5, 4, 3, 7, 1, 8, 6, 10], 'pasta': [1, 2, 2, 10, 2, 8, 6, 9, 7, 2], 'beef_noodle': [3, 8, 6, 7, 2, 9, 4, 10, 1, 5], 'pork_rice': [2, 8, 8, 8, 3, 4, 1, 6, 4, 7]}, 'ifoodie': {'ramen': [10, 8, 3, 1, 4, 6, 4, 6, 8, 1], 'hotpot': [6, 3, 5, 4, 2, 10, 1, 9, 7, 8], 'pasta': [3, 4, 5, 10, 1, 8, 2, 9, 6, 6], 'beef_noodle': [3, 8, 5, 9, 2, 7, 6, 9, 1, 4], 'pork_rice': [2, 10, 8, 9, 2, 4, 1, 7, 4, 6]}}


In [162]:
#====== choose food kind to get mse ======
def get_evaluation_result(food_kind, rank_dict, new_rating_score, store_names, all_store_post_count):
    print('\n===================== '+ food_kind +' rank =====================')
    store_rank = []
    temp_rating = new_rating_score[:]
    temp_rating.sort(reverse = True)
    cnt = 1
    for name,i in zip(store_names[1:],new_rating_score):
        rank = temp_rating.index(i)+1
        store_rank.append(rank)
        print('第'+str(cnt)+'名:',name,round(i,5),',',all_store_post_count[cnt-1],'post',',',rank)
        cnt += 1


    new_mse = MSE(np.array(range(1,11)),np.array(store_rank))
    print('\n新評分機制排名:',store_rank)
    print('MSE:',new_mse)
    print('r2_score:',round(r2_score(np.array(range(1,11)),np.array(store_rank)),3))
    print('--------------------Google--------------------')
    print('Google_rank: ' + str(rank_dict['google'][food_kind]))
    print('Google_MSE:',MSE(np.array(range(1,11)),np.array(rank_dict['google'][food_kind])))
    print('Google_r2_score:',round(r2_score(np.array(range(1,11)),np.array(rank_dict['google'][food_kind])),3))
    print('--------------------ifoodie--------------------')
    print('ifoodie_rank: ' + str(rank_dict['ifoodie'][food_kind]))
    print('ifoodie_MSE:',MSE(np.array(range(1,11)),np.array(rank_dict['ifoodie'][food_kind])))
    print('ifoodie_r2_score:',round(r2_score(np.array(range(1,11)),np.array(rank_dict['ifoodie'][food_kind])),3))
    return new_mse

# Cal one store rating

In [171]:
all_comments_scores

[1,
 1,
 1,
 0.0,
 0.2260699,
 0.16,
 0.11699999999999999,
 1,
 0.6796968,
 0.43113745000000003,
 0.56773935,
 1,
 0,
 1,
 0,
 0.0,
 1,
 1,
 0,
 0.7130968000000001,
 0,
 1,
 0.1,
 0,
 0.8229575,
 0.3957267,
 1,
 0.1382208,
 0.2,
 0.91,
 -0.3926862,
 1,
 0,
 0.2,
 0,
 -1,
 1,
 0.43700000000000006,
 1,
 0.863,
 0,
 0.52,
 1,
 0.32,
 1,
 1,
 0,
 -0.1,
 1,
 0,
 0.9279999999999999,
 1,
 1,
 1,
 0.535,
 1,
 0,
 0.0,
 1,
 1,
 0.99293045,
 1,
 0.2522301,
 0.6765839,
 1,
 1,
 0.37684395,
 0.2,
 1,
 0.75,
 1,
 0,
 0.02585350000000003,
 0.368,
 0.6881992,
 1,
 1,
 0.49678180000000005,
 0.62907405,
 0,
 1,
 0.2,
 1,
 0,
 0.0,
 0,
 0.6379814,
 1,
 1,
 1,
 -0.5214501,
 1,
 1,
 1,
 1,
 1,
 -0.1,
 0.57684395,
 -0.875,
 0.6920000000000001,
 1,
 0.034559549999999994,
 1,
 1,
 0,
 0.24171694999999999,
 1,
 0.3182022,
 0]

In [173]:
def norm_score_1(all_comments_scores):
    '''bigger or less than 1/-1 => 1/-1'''
    norm_all_comments_scores = []
    for one_score in all_comments_scores:
        if one_score >= 1:
            norm_all_comments_scores.append(1)
        elif one_score <= -1:
            norm_all_comments_scores.append(-1)
        else:
            norm_all_comments_scores.append(one_score)

    return norm_all_comments_scores

def norm_score_2(x):
    '''max_min norm to [-1,1]'''
    return [(float(i)-np.mean(x))/(max(x)-min(x)) for i in x]

def max_min_normalization(x):
    '''max_min norm to [0,1]'''
    return [(float(i)-min(x))/float(max(x)-min(x)) for i in x]

In [154]:
#====== cal one store rating ======

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score)
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


# get articles, followers
fileName = 'pasta/article_MINT Pasta 義大利麵' #
# fileName = 'pasta/article_螺絲瑪莉義麵坊' # 168
# fileName = 'pasta/article_村口微光' # 180


# fileName = 'hotpot/article_問鼎麻辣鍋忠孝店' # 147
# fileName = 'hotpot/article_滿堂紅bellavita店_松江店' # 101


# fileName = 'article_山嵐拉麵_公館店_台灣總店' # 85 post
# fileName = 'article_山嵐拉麵' # 33 post (remove)
# fileName = 'article_你回來啦 Okaeri 拉麵'# 96
# fileName = 'article_鬼金棒_中山北路' # 126
# fileName = 'article_麵屋壹の穴 ICHI' # 50 (remove)
# fileName = 'article_麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店' # 106
# fileName = 'article_柑橘 Shinn' # 69


articles = pd.read_csv(open('dataset/ig_article/store_article/'+fileName+'.csv')) 

# remove '\n', delete non-string
all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

# get each comment's score
all_comments_scores = get_each_comment_score(all_comments,True)
print(all_comments_scores)

# normalized all comment's senti_score
all_comments_scores = norm_score_1(all_comments_scores)
# all_comments_scores = norm_score_2(all_comments_scores)
# print(all_comments_scores)


# get store rating
# print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
store_rating = get_store_score(likes, all_comments_scores, assigned_index)
print('\nstore final score:',store_rating) # weight: likes
print('post count:',len(all_comments_scores))






comment: 這個時間最適合放食物照了😘😘，#mint #學生最愛 #平價美食 ，#米蘭茄汁海鮮義大利麵 💰175元，海鮮份量頗多，義大利麵本人也蠻入味，重點是cp值很高啊啊啊😘，，#台北美食 #台湾 #義大利麵 #pasta #lifestyle #foodlover

comment: 思思念念♥，終於吃到你😋😋

comment: 新北市 綠薄荷義大利麵🍝，，CP值很高，深受學生喜愛。，香辣海鮮墨魚麵NT$180，蘑菇濃湯NT$40，建議是早點到避免越晚越多人。，，#台灣生活 #台灣美食 #台湾旅行 #台湾美食 #台湾グルメ #台湾生活 #吃貨人生 #吃貨 #義大利麵 #パスタ #綠薄荷 #景美 #景美夜市 #taiwan #taiwanfood #pasta #gourmet #mintpasta #mintpasta義大利麵

comment: 文青看起來都很高，很用力吸肚子

comment: 粉紅醬起司雞肉麵 原本有微辣（很微），同桌點微辣的朋友覺得會吃膩，但我覺得剛好！，（我點跟我一樣辣度：小辣🌶️），建議吃之前要先攪拌，，不然辣都在底層～越吃越辣🌶️🌶️🌶️

comment: 綠蓉蛤蠣，#義大利麵 #青醬 #青醬義大利麵 #mintpasta #pasta #oyster #basilpasta #景美美食 #taipei #taiwan

comment: 🍝

comment: Mint竟然多了新餐點薯條，紅黃綠三種醬可以選，很像綠蓋的味道，好吃。，奶油煙燻鮭魚的口味超好吃，但不知道為什麼廚師常常漏掉這個口味的調味，已經是第二次了。，最近照片越發越慢了XD

comment: 感謝兩位朋友請吃飯😋，#酥皮南瓜濃湯 ，#燻鴨義大利麵

comment: 世新必吃的義大利麵，然後我實踐的，·，·，·，#taiwan #taipei #igdaily #daily #dailylfe #flatlay #foodporn #yummy #foodie #igtw #foodstagram #instafood #oishi #foods #foodphotography #foody #taiwanesefood #lunch #spaghetti #pasta #pesto ，#台灣 #台北 #午餐 #義大利麵 #相機食先

comment

# Cal one food kind rating

In [163]:
#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict

#====== select food kind to get store names ======
for select_food_kind in ['pork_rice']:
    store_names = get_store_names(select_food_kind)

    # cal_all_store_rating
    all_store_rating, all_store_post_count = cal_all_store_rating(store_names,False) # data_preprocess: pick at most post
    # evaluation
    get_evaluation_result(select_food_kind, rank_dict, all_store_rating, store_names, all_store_post_count)



===================== pork_rice rank =====================
第1名: 今大 0.17623 , 150 post , 9
第2名: 金峰魯肉飯 0.27863 , 150 post , 5
第3名: 萬華阿義魯肉飯 0.35022 , 150 post , 4
第4名: 山河魯肉飯 麻薏湯 0.44536 , 111 post , 1
第5名: 金澤魯肉飯-幸福宜蘭店 0.18295 , 116 post , 8
第6名: 禾日香-魯肉飯專門店 0.41599 , 82 post , 2
第7名: 痣男 0.22603 , 90 post , 6
第8名: 南豐魯肉飯 0.35926 , 111 post , 3
第9名: 小王清湯瓜仔肉 0.2029 , 123 post , 7
第10名: 矮仔財滷肉飯 0.15024 , 112 post , 10

新評分機制排名: [9, 5, 4, 1, 8, 2, 6, 3, 7, 10]
MSE: 13.8
r2_score: -0.673
--------------------Google--------------------
Google_rank: [2, 8, 8, 8, 3, 4, 1, 6, 4, 7]
Google_MSE: 16.0
Google_r2_score: -0.939
--------------------ifoodie--------------------
ifoodie_rank: [2, 10, 8, 9, 2, 4, 1, 7, 4, 6]
ifoodie_MSE: 20.6
ifoodie_r2_score: -1.497


# Cal all food kind rating

In [170]:
#====== MSE ======
# input store name: [a,b,c]
# output: [score.....]
def cal_all_store_rating(store_names, print_comment = True):
    all_store_rating = []
    all_store_post_count = [] # store info dict.
    for i,name in enumerate(store_names[1:]):
#         print('-----------------------------------')
#         print(name)
        fileName = '/article_' + name
        articles = pd.read_csv(open('dataset/ig_article/store_article/'+store_names[0]+fileName+'.csv')) 

        # remove '\n', delete non-string
        all_comments, assigned_index, fans_count, likes = data_preprocess(articles)  

        # get each comment's score
        all_comments_scores = get_each_comment_score(all_comments,print_comment)
        
        # normalize senti score
#         all_comments_scores = norm_score_1(all_comments_scores)
#         all_comments_scores = norm_score_2(all_comments_scores)
        all_comments_scores = max_min_normalization(all_comments_scores)
        
        # get store rating
        # print('\nstore final score:',get_store_score(fans_count, all_comments_scores, assigned_index)) # weight: fans
        store_rating = get_store_score(likes, all_comments_scores, assigned_index)
        
#         print('\nstore final score:',store_rating) # weight: likes
        all_store_rating.append(store_rating)
        all_store_post_count.append(len(all_comments))
        
    return all_store_rating,all_store_post_count



#===================================== main function =======================================

#====== update senti_dict & jieba_dict ======
self_extend_senti_dict(word_senti_score) 
jieba.load_userdict('dataset/dic_trad/self_define_dict.txt') # load self-define dict


#====== select food kind to get store names ======
mse_record = {}
for select_food_kind in ['ramen','hotpot','pasta','beef_noodle','pork_rice']:
    store_names = get_store_names(select_food_kind)

    # cal_all_store_rating
    all_store_rating, all_store_post_count = cal_all_store_rating(store_names,False) # data_preprocess: pick at most post
    # evaluation matrix
    new_mse = get_evaluation_result(select_food_kind, rank_dict, all_store_rating, store_names, all_store_post_count)
    mse_record[select_food_kind] = new_mse

print('\n'+str(mse_record))


===================== ramen rank =====================
第1名: 鬼金棒_中山北路 0.18062 , 125 post , 6
第2名: 雞吉君拉麵 0.3324 , 47 post , 2
第3名: 山嵐拉麵_公館店_台灣總店 0.11046 , 85 post , 10
第4名: 你回來啦 Okaeri 拉麵 0.16834 , 95 post , 8
第5名: 麵屋壹慶 0.17748 , 65 post , 7
第6名: 勝王 0.13636 , 142 post , 9
第7名: 五之神製作所 0.38242 , 101 post , 1
第8名: 柑橘 Shinn 0.27943 , 69 post , 3
第9名: 麵屋壹の穴 ICHI_麵屋壹の穴ichi-沾麵專門店 0.197 , 106 post , 5
第10名: 山下公園 ラーメン 0.2729 , 134 post , 4

新評分機制排名: [6, 2, 10, 8, 7, 9, 1, 3, 5, 4]
MSE: 21.6
r2_score: -1.618
--------------------Google--------------------
Google_rank: [10, 4, 6, 4, 1, 7, 7, 1, 9, 1]
Google_MSE: 24.1
Google_r2_score: -1.921
--------------------ifoodie--------------------
ifoodie_rank: [10, 8, 3, 1, 4, 6, 4, 6, 8, 1]
ifoodie_MSE: 22.2
ifoodie_r2_score: -1.691

===================== hotpot rank =====================
第1名: 辛殿麻辣鍋 0.22869 , 150 post , 4
第2名: 海底撈火鍋 0.14543 , 150 post , 8
第3名: 老四川巴蜀麻辣燙-南京店 0.22703 , 150 post , 5
第4名: 這一鍋皇室秘藏鍋物 0.23749 , 114 post , 3
第5名: 詹記麻辣火鍋 敦南店 0.27928

In [43]:
mae(np.array(range(1,11)),np.array([9,2,5,4,3,7,1,8,6,10]))

# output ramen rating 
# output_csv('ramen_rating.csv', [[fileName, store_rating]], 'a+')

2.2

# Scrape url via selenium

In [155]:
store_selenium_param = get_selenium_param('selenium_url')
store_names = []
store_page_xpath = []
first_page_xpath = []

for line in store_selenium_param:
    store_names.append(line[0])
    store_page_xpath.append(line[1])
    first_page_xpath.append(line[2])

    
print(first_page_xpath)
# store_names = [store_selenium_param[0][0]]
# store_page_xpath = [store_selenium_param[0][1]]
# first_page_xpath = [store_selenium_param[0][2]]
# get_page_count = [4000]
# print(first_page_xpath)

['href="/p/B_PgFUaDk2y/"', 'href="/p/B_RlyLqA0z8/"', 'href="/p/B_SHtKjJof5/"']


In [156]:
#===== scrape url from first post ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(3)

    # get url of current page
    url_list, post_date = [], []
    url_list.append([driver.current_url]) # get first page url
    time.sleep(1)
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
    time.sleep(3)

    # get 500 page url
#     flag = 0
#     for i in range(get_page_count[store_index]):
    for i in range(8000):
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click()
        else:
            break

        time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(5)
        post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
        stopDate = post_date[-1][0]
#         print(post_date[-1][0],type(post_date[-1][0]))
        
        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list, 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], post_date, 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print(stopTime-startTime/60)

HUN 混finished!
MINT Pasta 義大利麵finished!
JAI 宅finished!
1561167483.6183445


In [40]:
#===== scrape url from assigned date ======
startTime = time.time()

#====== use selenium to crawl store url ======
from selenium import webdriver

ig_home = "https://instagram.com"
driver = webdriver.Chrome(executable_path="chromedriver_linux64/chromedriver") # Use Chrome
driver.get(ig_home) # open ig home page
time.sleep(2)

# login
driver.find_element_by_xpath('//input[@name="username"]').send_keys('bestronger10142008')
driver.find_element_by_xpath('//input[@name="password"]').send_keys('igcoke1231')
time.sleep(2)
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

# iterate to get each store url
for store_index in range(len(store_names)):
    # input store name
    driver.find_element_by_xpath('//input[@type="text"]').send_keys(store_names[store_index]) 
    time.sleep(4)
    
    # driver.find_element_by_xpath('//a[@href="/explore/tags/雞吉君拉麵/"]').click() # click store page
    driver.find_element_by_xpath('//a[@' + store_page_xpath[store_index] + ']').click() # click store page
    time.sleep(5)
    
    # driver.find_element_by_xpath('//a[@href="/p/B-CEZjVl8GC/"]').click() # click first page link
    driver.find_element_by_xpath('//a[@' + first_page_xpath[store_index] + ']').click() # click first page link
    time.sleep(4)

    # get url of current page
    url_list, post_date, output_post_date = [], [], []
    post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]) # get first page date

    # get 500 page url
    for i in range(5000):
        
        stopDate = post_date[-1][0]
        
        if stopDate[:stopDate.find('年')] != '2019':
#             print(stopDate)
            
            if check_exists_by_xpath('//a[@tabindex="0"][2]'):
                driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
                time.sleep(3)
            else:
                break
            time.sleep(1)
            post_date.append([driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text])
            continue
            
            
#         time.sleep(1)
        url_list.append([driver.current_url])
        time.sleep(1)
        temp_date = [driver.find_element_by_xpath('//time[@class="_1o9PC Nzb55"]').text]
        post_date.append(temp_date)
        output_post_date.append(temp_date)
        
        if check_exists_by_xpath('//a[@tabindex="0"][2]'):
            driver.find_element_by_xpath('//a[@tabindex="0"][2]').click() # click next page
            time.sleep(4)
        else:
            break

        # stop date = 2019.6.1
        if stopDate[:stopDate.find('年')] == '2019' and stopDate[stopDate.find('年')+1:stopDate.find('月')] == '5':
            break
        
            
    ActionChains(driver).send_keys(Keys.ESCAPE).perform() # close floating page
    output_csv('dataset/ig_article/store_url/url_' + store_names[store_index], url_list[:-1], 'w+')
    output_csv('dataset/ig_article/store_url/date_' + store_names[store_index], output_post_date[:-1], 'w+')
    print(store_names[store_index]+'finished!')
    
driver.close()
stopTime = time.time()
print((stopTime-startTime)/60)

新馬辣武昌店finished!
新馬辣經典馬辣鍋finished!
青花驕麻辣鍋板橋縣民大道店finished!
馬辣頂級麻辣鴛鴦火鍋-中山店finished!
馬辣頂級麻辣鴛鴦火鍋 漢口店finished!
問鼎麻辣鍋忠孝店finished!
問鼎 麻辣鍋 養生鍋 粉絲俱樂部finished!
滿堂紅-松江店finished!
滿堂紅bellavita店finished!
153.04108297030132


In [259]:
# teststr = '2019年8月10日'
teststr = '3月23日'
if teststr[:teststr.find('年')] == '2019' and teststr[teststr.find('年')+1:teststr.find('月')] == '8':
    print('yes')

# Extend antusd

In [260]:
#====== word segment ======
all_reviews = []
for filename in os.listdir('mac_extended_dict/google_review/'):              # read every file
    with open('mac_extended_dict/google_review/'+filename, newline='') as f: # open in readonly mode
        rows = csv.reader(f)
        
        firstline = True
        for post in rows:           # read each user post
           
            if firstline == True:   # skip fisrt line
                firstline = False
                continue
            tmp_star = int(post[2].strip()[0])
            tmp_review = post[3].strip().replace('(由 Google 提供翻譯)','').replace('(原始評論)','')
            if tmp_review == '':    # skip empty review
                continue
            all_reviews.append([tmp_star,tmp_review])
            

all_reviews = np.array(all_reviews)



In [267]:
def remove_punctuation(line):
    punctuation = "$,~-:!。; ！？｡＂＃＄％＆＇（）＊＋，－/\／：；＜＝＞#＠［＼］＾＿｀｛｜｝～()｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."
    punctuation = [i for i in punctuation] + ['\n','--','☆','⋯','..']
    line = [word for word in line if word not in punctuation and not word.isdigit()]

    return line

In [268]:
for i in all_reviews[:,1]:
    print(remove_punctuation(jieba.lcut(i)))

['兩年', '前', '在', '這裡', '辦', '婚禮', '美食', '沒', '能', '好好', '品嚐', '今天', '帶', '老婆', '回來', '細細', '咀嚼', '真的', '是', '很棒', '的', '一家', '餐廳', '環境', '和', '氣氛', '很', '到位', '適合', '情侶', '和', '家族', '聚餐', '👍', '感覺', '的', '出來', '食材', '特別', '新鮮', '健康', '超級', '滿足', '👍', '👍', '對於', '麵', '包', '特別', '挑剔', '的', '我', '迷上', '它', '的', '麵', '包', '我怕', '以後', '吃', '不到', '怎麼辦', '阿', '😆', '😆']
['台大', '校園內', '高級', '西餐廳', '環境', '安靜', '優雅', '座位', '寬敞', '義大利', '菜', '健康', '取向', '餐點', '取材', '用心', '值得', '推薦']
['餐點', '令人', '驚艷', '用餐', '環境', '舒適', '愜意', '適合', '聚會', '或是', '家庭', '用餐', '值得', '推薦']
['煙燻', '鴨腿', '肋', '眼心']
['隱藏', '在', '台大', '校園', '裡的', '義式', '餐廳', '空間', '寬敞', '舒適', '餐點', '好吃', '有', '創意', '服務', '人員', '親切', '又', '專業']
['素菜', '也', '做', '得', '精緻', '好吃', '服務', '週到', '用餐', '環境', '安靜', '且', '氣氛', '良好']
['適合', '帶', '女友', '來', '的', '好', '地方', '我', '當天', '拍照', '寫實', '不', '刻意', '過', '曝', '美化']
['很', '用心', '的', '訂製', '式', 'Buffet', '食材', '味道', '價錢', '一流']
['所有', '的', '東西', '都', '很', '有', '品質', '這次', '我點', '了', '松阪豬', '令人', '驚

['麵', '體', '很好吃', '可加辣', '有', '畫龍點睛', '的', '效果']
['這', '真是', '我吃過', '最', '難吃', '的', '日本', '拉麵', '了', '完全', '不', '值得', '排隊', '啊']
['半', '夜來', '吃', '完全', '不需', '排隊', '等候', '服務', '人員', '態度', '良好', '麵', '彈性', '和', '湯頭', '和', '勾選', '的', '如出一轍', '好吃']
['服務', '很', '用心', '拉麵', '也', '很好吃']
['深夜', '的', '好', '選擇', '晚點', '去', '比較', '好', '大概', '排', '分', '吧', '很快', '就', '到', '了', '細', '麵', '就是', '討人喜歡', '啊', '好吃']
['湯頭', '濃郁', '麵', '條有', '嚼', '勁', '叉燒', '好吃', '價格', '有點', '貴', '但', '一人', '一個', '坐位', '頗', '特別', '的']
['湯和', '麵', '條', '好吃', '味道', '與', '日本', '差不多', '不過', '肉', '真的', '完全', '不行', '以', '價位', '和', '排隊', '等待', '的', '時間', '來說', '不太值', '五顆星']
['偶爾', '吃', '吃還', '不錯', '麵', '條吃', '完剩', '的', '湯配', '上', '店裡', '賣', '的', '飯', '還頗', '好吃', '真心', '推薦', '要', '加點', '白飯']
['吃', '起來', '普通', '不過', '常常', '大排長龍']
['真的很好吃', '隨時', '都', '可以', '吃', '兩碗']
['平日', '的', '一蘭', '終於', '沒有', '大排長龍', '偶爾', '還是', '會', '想念', '日本', '的', '味道', '快來', '解解饞']
['難得', '會', '讓', '我', '對於', '拉麵', '很', '挑', '的', '母親', '稱讚', '的', '店', '雖

['深藏', '在', '一中', '街', '附近', '的', '大樓', '裡', '卻是', '人來人往', '的', '打開', '菜單', '不', '簡單', '的', '來', '了', '頁', '飯', '麵', '炸物', '甜點', '早', '午餐', '調酒', '啤酒', '飲料', '薄餅', '想當然', '爾', '只好', '請', '服務生', '幫忙', '介紹', '橘醬', '炸', '蝦', '麵', '酸酸甜甜', '的', '義大利', '麵', '配上', '尾炸', '蝦', '堪稱', '絕品', '這橘', '味', '不是', '橘子', '生成', '而是', '綜合', '水果', '調味', '而成', '是', '本店', '特色', '醬汁', '明太子', '烤雞肉', '串', '實在太', '厲害', '上面', '微', '甜醬', '汁', '及', '焦度', '恰到好處', '的', '鮮嫩', '來', '一份', '絕對', '滿足', '👍']
['位', '在', '光南', '二樓', '小米', '門市', '隔壁', '尖峰', '用餐', '時間', '限制', '分鐘', '假日', '一定', '要', '先', '訂位', '不然', '要', '等', '很', '久', '內部', '環境', '綠意盎然', '真假', '植物', '交錯', '擺設', '每個', '空間', '和', '角落', '都', '具', '巧思', '佈置', '功力', '了', '得', '環境優美', '餐點', '也', '不', '遜色', '意大利', '麵', '有', '麵', '心', '的', '口感', '富嚼', '勁', '但', '不會', '太', '生硬', '橘醬', '蛤蜊', '麵', '滿滿的', '蛤蜊', '讓', '你', '吃', '到', '懷疑', '人生', '吃', '完後尖', '尖', '的', '蛤蜊', '殼', '很', '有', '成就感', '明太子', '烤雞', '串', '麵', '令人', '非常', '驚豔', '碳', '烤雞', '串', '加上', '明太子', '醬', '太', '銷

['網', '美名', '店', '店內', '佈置', '用心', '有', '很多', '好看', '的', '打卡', '裝飾', '座位', '排', '得', '不會', '太擠', '吃', '起來', '沒有', '壓迫感', '沒', '訂位', '等', '了', '快', '一', '小時', '不過', '等待', '還蠻', '值得', '的', '這次', '點的', '是', '橘醬', '鮮蝦', '義大利', '麵', '跟', '泡菜', '雞肉', '義大利', '麵', '價位', '都', '在', '之間', '加', '元', '套餐', '有', '紅茶', '跟', '濃湯', '紅茶', '小小', '一杯', '偏甜', '濃', '湯料', '很', '豐富', '有', '馬鈴薯', '蛤蜊', '又', '有', '培根']
['餐點', '略貴', '不', '提供', '刷卡', '或', '行動', '支付']
['蠻', '不錯', '的', '環境', '寬敞', '舒適']
['口味', '偏重', '但是', '好吃', '的', '菜單', '的', '選項', '真的', '多到', '眼花撩亂', '有些', '取名', '其實', '也', '看', '不', '太', '懂', '是', '什麼', '口味', '譬如', '普羅旺斯', 'XX', '麵', '所以', '是', '有', '薰衣草', '在', '裡面', '嗎', '?', '或是', 'XXX', '宅宅', '麵', '所以', '是', '寬', '麵', '細', '麵', '還是', '斜管', '麵', '?', '但', '始終', '沒', '搞懂', '因為', '直接', '去點', '看得懂', '的', '就', '好', '了', '😂', '價位', '是', '合理', '的', '分量', '也', '頗', '大', '的', '唯一', '的', '建議', '就是', '希望', '可以', '在', '菜單', '上', '解釋一下', '特定', '的', '義大利', '麵', '的', '醬', '烹煮', '方式', '會', '減少', '客人', '的', '困

# combine csv

In [317]:
article1 = pd.read_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA.csv')
article2 = pd.read_csv('dataset/ig_article/store_article/pasta/article_米家廚房.csv')
combined_article = pd.concat([article1, article2], ignore_index=True)
combined_article.to_csv('dataset/ig_article/store_article/pasta/article_MIGA KITCHEN PASTA_米家廚房.csv')

# extend dictionary

In [413]:
# extend dictionary
word_to_weight = {
    "土地公": 1,
    "土地婆": 1,
    "公有": 2,
    "緯來體育台": 1,
    "鬼金棒": 3,
    "海底撈": 5
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(2, {'公有': 2.0}), (3, {'土地公': 1.0, '土地婆': 1.0, '鬼金棒': 3.0, '海底撈': 5.0}), (5, {'緯來體育台': 1.0})]
